Notebook purpose

- Replicate analysis of davenport2020spending

In [3]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [24]:
d = aws.S3BucketManager('3di-project-entropy')
d.list()

['3di-project-entropy/entropy_000.parquet',
 '3di-project-entropy/entropy_777.parquet']

In [25]:
df = aws.s3read_parquet('s3://3di-data-mdb/raw/mdb_000.parquet')


In [51]:
SAMPLE = '777'
fp = f'~/tmp/entropy_{SAMPLE}.parquet'

df = aws.read_parquet(fp)
df.head(2)

,id,date,user_id,amount,desc,merchant,tag_up,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_recorded_balance,merchant_business_line,tag,tag_auto,tag_manual,ym
0,20201743,2014-02-24,58777,10.0,lnk po woodgate 04 cd 2813 22feb14,NaN,cash,True,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-10-23,True,8.38,personal,None,cash,NaN,201402
1,20201744,2014-02-24,58777,-5.0,paypal 22 <mdbremoved>,paypal,paypal account,True,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-11-13,False,8.38,paypal,None,enjoyment,NaN,201402


In [52]:
tags = ['tag', 'tag_up', 'tag_manual', 'tag_auto']
df[tags]

,tag,tag_up,tag_manual,tag_auto
0,None,cash,NaN,cash
1,None,paypal account,NaN,enjoyment
2,None,cash,NaN,cash
3,None,cash,NaN,cash
4,None,transfers,NaN,transfers
...,...,...,...,...
12671,None,"food, groceries, household",NaN,"food, groceries, household"
12672,None,"medical, dental, eye care",NaN,"medical, dental, eye care"
12673,None,credit card,NaN,credit card
12674,None,enjoyment,NaN,enjoyment


In [54]:
df.tag

0        None
1        None
2        None
3        None
4        None
         ... 
12671    None
12672    None
12673    None
12674    None
12675    None
Name: tag, Length: 12676, dtype: object

In [53]:
cl.fill_tag(df[tags])

/Users/fgu/dev/projects/entropy/entropy/data/cleaners.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tag'] = df.tag.where(df.tag.notna(), df.tag_up)


,tag,tag_up,tag_manual,tag_auto
0,cash,cash,NaN,cash
1,paypal account,paypal account,NaN,enjoyment
2,cash,cash,NaN,cash
3,cash,cash,NaN,cash
4,transfers,transfers,NaN,transfers
...,...,...,...,...
12671,"food, groceries, household","food, groceries, household",NaN,"food, groceries, household"
12672,"medical, dental, eye care","medical, dental, eye care",NaN,"medical, dental, eye care"
12673,credit card,credit card,NaN,credit card
12674,enjoyment,enjoyment,NaN,enjoyment


In [43]:
tag_names = ['tag', 'tag_up', 'tag_manual', 'tag_auto']
tags = df[tag_names]

mask = tags['tag_up'].isna() & (tags['tag_auto'].notna() | tags['tag_manual'].notna())
errors = tags[mask]
errors.head(3)

,tag,tag_up,tag_manual,tag_auto


In [44]:
tag_names = ['tag', 'tag_up', 'tag_manual', 'tag_auto']
tags = df[tag_names]

mask = tags['tag_up'].notna() & tags['tag_auto'].isna() & tags['tag_manual'].isna()
errors = tags[mask]
errors.head(3)

,tag,tag_up,tag_manual,tag_auto
4995,None,credit card,NaN,NaN
4996,None,credit card,NaN,NaN
5003,None,credit card,NaN,NaN
